In [1]:
import pandas as pd

generate='./data/generate/'
path='./data/rec_data/'

In [17]:
train=pd.read_csv(generate+'train_all022.csv')
test=pd.read_csv(generate+'test_new022.csv')
data=pd.concat([train,test])

In [2]:
user_seq=pd.read_csv(generate+'user_seq_raw.csv',usecols=['suv','itemId','length'])
category=pd.read_csv(generate+'category.csv')
category.rename(columns={"id":"itemId"},inplace=True)
user_seq=pd.merge(user_seq,category,how='left',on='itemId')

In [3]:
user_seq['item_count'] = user_seq.groupby('itemId')['suv'].transform('count')

In [4]:
def fun(x):
    if x<=500:
        return 1
    elif x<=2000:
        return 2
    elif x<=6000:
        return 3
    else:
        return 4

In [5]:
user_seq['level']= user_seq['item_count'].apply(lambda x: fun(x))

In [10]:
hist_click=user_seq[['suv','itemId','category','user_level','length','item_count','level']].groupby(['suv','length','user_level']).agg({list}).reset_index()
his_behavior_df=pd.DataFrame()
his_behavior_df['suv']=hist_click['suv']
his_behavior_df['hist_itemId']=hist_click['itemId']
his_behavior_df['hist_category']=hist_click['category']
his_behavior_df['hist_item_count']=hist_click['item_count']
his_behavior_df['hist_item_level']=hist_click['level']
his_behavior_df['user_level']=hist_click['user_level']
his_behavior_df['length']=hist_click['length']

In [11]:
tmps=his_behavior_df['hist_itemId'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
    tmps[i]=tmps[i]+[0]*(21-len(tmps[i]))
his_behavior_df['hist_itemId']=tmps

tmps=his_behavior_df['hist_category'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
    tmps[i]=tmps[i]+[0]*(21-len(tmps[i]))
his_behavior_df['hist_category']=tmps

tmps=his_behavior_df['hist_item_count'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
    tmps[i]=tmps[i]+[0]*(21-len(tmps[i]))
his_behavior_df['hist_item_count']=tmps

tmps=his_behavior_df['hist_item_level'].values.tolist()
for i in range(len(tmps)):
    if not isinstance(tmps[i],list):
        tmps[i]=[0]
    tmps[i]=tmps[i]+[0]*(21-len(tmps[i]))
his_behavior_df['hist_item_level']=tmps

In [12]:
hist_itemId = his_behavior_df['hist_itemId'].apply(lambda x: pd.Series(x))
hist_itemId.columns = ['hist_itemId{}'.format(i) for i in range(21)]
his_behavior_df = pd.concat([his_behavior_df, hist_itemId], axis=1)

hist_category = his_behavior_df['hist_category'].apply(lambda x: pd.Series(x))
hist_category.columns = ['hist_category{}'.format(i) for i in range(21)]
his_behavior_df = pd.concat([his_behavior_df, hist_category], axis=1)

hist_item_count = his_behavior_df['hist_item_count'].apply(lambda x: pd.Series(x))
hist_item_count.columns = ['hist_item_count{}'.format(i) for i in range(21)]
his_behavior_df = pd.concat([his_behavior_df, hist_item_count], axis=1)

hist_item_level = his_behavior_df['hist_item_level'].apply(lambda x: pd.Series(x))
hist_item_level.columns = ['hist_item_level{}'.format(i) for i in range(21)]
his_behavior_df = pd.concat([his_behavior_df, hist_item_level], axis=1)

In [14]:
del his_behavior_df['hist_itemId']
del his_behavior_df['hist_category']
del his_behavior_df['hist_item_count']
del his_behavior_df['hist_item_level']

In [19]:
data=pd.merge(data,his_behavior_df,how='left',on='suv')

In [22]:
data.to_feather(generate+'data_all023.feather')